In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from geopy.geocoders import Nominatim

/Users/maxdonheiser/.pyenv/versions/3.9.1/lib/python3.9/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.3-CAPI-1.16.1). Conversions between both will be slow.
  warnings.warn(


In [2]:
de = gpd.read_file("/Users/maxdonheiser/Documents/DE_Geodaten/kreisgrenzen.geojson")

In [3]:
fpath = "frauenhaus_suche.json"

# load data
raw = pd.read_json(fpath, orient="index")
raw["shelter_id"] = raw.index
raw = raw.reset_index(drop=True)

# clean up geometry columns
raw.loc[raw.geography.notnull(), "geometry"] = raw.loc[raw.geography.notnull(), "geography"]

# get all timeseries data
df = pd.DataFrame()

for i, row in raw.iterrows():
    
    temp = pd.DataFrame(row["data"])
    temp["shelter_id"] = row["shelter_id"]
    temp["shelter_name"] = row["title"]
    temp["latitude"] = row["geometry"]["coordinates"][1]
    temp["longitude"] = row["geometry"]["coordinates"][0]
    df = pd.concat([df, temp])
    
df = df[["shelter_name","shelter_id","latitude","longitude"]].drop_duplicates()

df = df.loc[df.shelter_name.notnull(),]

df["points"] = gpd.points_from_xy(df.longitude, df.latitude, crs="EPSG:4326")

df["gen"] = np.nan
df["bez"] = np.nan
df["nuts"] = np.nan

df = df.reset_index(drop=True)

In [4]:
# geocode kreis
for j, row in df.iterrows():
    
    point = row["points"]
    shelter = row["shelter_name"]
    shelter_id = row["shelter_id"]
    found = False
    
    for k, kreis in de.iterrows():
        
        geometry = kreis["geometry"]
        
        if geometry.contains(point):
            
            df.loc[j, "bez"] = kreis["BEZ"]
            df.loc[j, "gen"]  = kreis["GEN"]
            df.loc[j, "nuts"]  = kreis["NUTS"]
            found = True
            continue
    
    if found == False:
        print(f"Failed on {shelter}, {shelter_id}")

Failed on AWO Frauen- und Kinderschutzhaus Bodenseekreis, 2333
Failed on Frauenhaus Kiel, 2106
Failed on AWO Frauenschutzhaus Ribnitz-Damgarten, 2205


In [5]:
geolocator = Nominatim(user_agent="correctiv")

for i, row in df.loc[df.bez.isnull()].iterrows():
    
    shelter = row["shelter_name"]
    shelter_id = row["shelter_id"]
    coordinates = f"{row['latitude']}, {row['longitude']}"
    address = geolocator.reverse(coordinates)
    print(address)

Hofen, Friedrichshafen, Verwaltungsgemeinschaft Friedrichshafen, Bodenseekreis, Baden-Württemberg, 88045, Deutschland
Kiel, Sartorikai, Altstadt, Kiel, Schleswig-Holstein, 24103, Deutschland
Spielplatz Hafen, Am See, Ribnitz, Ribnitz-Damgarten, Vorpommern-Rügen, Mecklenburg-Vorpommern, 18311, Deutschland


In [6]:
df.loc[df.shelter_id==2333,["gen","bez","nuts"]] = ["Bodenseekreis", "Kreis", "DE147"]
df.loc[df.shelter_id==2106,["gen","bez","nuts"]] = ["Kiel", "Kreisfreie Stadt", "DEF02"]
df.loc[df.shelter_id==2205,["gen","bez","nuts"]] = ["Vorpommern-Rügen", "Kreis", "DE80L"]

In [7]:
df.loc[df.gen=="Göttingen", "nuts"] = "DE91C"

In [8]:
# get bundesländer
states = gpd.read_file("/Users/maxdonheiser/Documents/DE_Geodaten/bundesländergrenzen.geojson")
states = states[["NUTS","GEN"]].rename(columns={"NUTS":"nuts2","GEN":"bundesland"})

In [9]:
# and merge
df["nuts2"] = df.nuts.str[0:3]
df = pd.merge(df, states, on="nuts2", how="outer")

In [10]:
df.head()

,shelter_name,shelter_id,latitude,longitude,points,gen,bez,nuts,nuts2,bundesland
0,Frauen- und Kinderschutzhaus Aachen,2270,50.788047,6.097588,POINT (6.09759 50.78805),Städteregion Aachen,Kreis,DEA2D,DEA,Nordrhein-Westfalen
1,Frauenhaus Alsdorf,2244,50.871672,6.208659,POINT (6.20866 50.87167),Städteregion Aachen,Kreis,DEA2D,DEA,Nordrhein-Westfalen
2,Frauenhaus Arnsberg,2182,51.407733,8.051076,POINT (8.05108 51.40773),Hochsauerlandkreis,Kreis,DEA57,DEA,Nordrhein-Westfalen
3,Frauenhaus Bergisch Gladbach,2067,50.992224,7.153430,POINT (7.15343 50.99222),Rheinisch-Bergischer Kreis,Kreis,DEA2B,DEA,Nordrhein-Westfalen
4,AWO Frauenhaus Bielefeld,2118,52.035302,8.526658,POINT (8.52666 52.03530),Bielefeld,Kreisfreie Stadt,DEA41,DEA,Nordrhein-Westfalen


In [11]:
df = df.drop(["points"], axis=1)

In [12]:
df.loc[df.bundesland.isnull()]

,shelter_name,shelter_id,latitude,longitude,gen,bez,nuts,nuts2,bundesland


In [13]:
df.to_csv("data/helpers/shelters_geocoded.csv", index=False)